<a href="https://colab.research.google.com/github/krisabundiz/Evidencias/blob/main/EVIDENCIA_3(CORREGIDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT CORRESPONDIENTES**

In [5]:
import openpyxl
import csv
from openpyxl import Workbook
from openpyxl.styles import Font
from datetime import datetime
import re
from prettytable import PrettyTable
import sys
import sqlite3
from sqlite3 import Error

**CREACIÓN DE TABLAS**

In [2]:
try:
    with sqlite3.connect('/content/tallermecanico.db') as conn:
        mi_cursor = conn.cursor()
        mi_cursor.execute('CREATE TABLE IF NOT EXISTS clientes (ClaveCliente INTEGER PRIMARY KEY, nombre TEXT NOT NULL, correo TEXT NOT NULL, rfc TEXT NOT NULL, estadoC BLOB NOT NULL);')
        mi_cursor.execute('CREATE TABLE IF NOT EXISTS servicios (ClaveServicio INTEGER PRIMARY KEY, descripcion TEXT NOT NULL, costo FLOAT NOT NULL, estadoS BLOB NOT NULL);')
        mi_cursor.execute('CREATE TABLE IF NOT EXISTS notas (folio INTEGER PRIMARY KEY, fecha timestamp NOT NULL, ClaveCliente INTEGER, monto REAL NOT NULL,estadoN BLOB NOT NULL ,FOREIGN KEY (ClaveCliente) REFERENCES clientes (ClaveCliente));')
        mi_cursor.execute('CREATE TABLE IF NOT EXISTS notas (ClaveCliente INTEGER, FOREIGN KEY (ClaveCliente) REFERENCES clientes (ClaveCliente), ClaveServicio INTEGER, FOREIGN KEY (ClaveServicio) REFERENCES servicios (ClaveServicios));')
        print('TABLAS CREADAS EXITOSAMENTE')
except Error as e:
    print(e)
except Exception:
    print(f'Se produjo el siguiente error: {sys.exc_info()[0]}')


near "ClaveServicio": syntax error


**MENÚ NOTAS**

In [6]:
def menu_notas():
  def registrar_nota():
    fecha= None
    while True:
        print('\n   REGISTRAR UNA NUEVA NOTA')
        print('\nNota: Escriba (salir) si desea volver al menú principal')
        try:
            with sqlite3.connect('C:/Users/betyh/Downloads/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveCliente, nombre FROM clientes')
                datos = mi_cursor.fetchall()
                tabla = PrettyTable()
                tabla.field_names = ["CLAVE", "NOMBRE"]
                print('\n    Clientes Registrados')
                for dato in datos:
                    tabla.add_row(dato)
                print(tabla)
        except Error as e:
            print(e)
        except Exception:
            print(f'Se produjo el siguiente error: {sys.exc_info()[0]}')
        finally:
            conn.close()
        while True:
            try:
                cliente = input('\nIngrese la clave del cliente: ')
                if cliente.lower() == 'salir':
                    print('\n** OPERACIÓN CANCELADA. VOLVIENDO AL MENÚ PRINCIPAL **')
                    return
                if cliente.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UNA DATO O (SALIR) PARA VOLVER AL MENÚ PRINCIPAL **')
                    continue
                else:
                   cliente = int(cliente)
                   with sqlite3.connect('C:/Users/betyh/Downloads/tallermecanico.db') as conn:
                        mi_cursor = conn.cursor()
                        mi_cursor.execute('SELECT ClaveCliente FROM clientes WHERE ClaveCliente = ?', (cliente,))
                        cliente_existente = mi_cursor.fetchall()
                   if cliente_existente:
                        break
                   else:
                       print('\n** EL CLIENTE NO ESTÁ REGISTRADO. POR FAVOR, REGISTRE AL CLIENTE ANTES DE CONTINUAR **')
                       continue
            except Exception:
                print(f'\n** DATO NO VÁLIDO. INGRESE LA CLAVE DE ALGÚN CLIENTE **')

        while True:
            try:
                fecha_str = input('\nIngresa la fecha en formato DD/MM/YYYY: ')
                if fecha_str.lower() == 'salir':
                    print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ PRINCIPAL **')
                    return
                if fecha_str.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UNA DATO O (SALIR) PARA VOLVER AL MENÚ PRINCIPAL **')
                    continue
                fecha_actual = datetime.now().date()
                fecha_ingresada = datetime.strptime(fecha_str, "%d/%m/%Y").date()
                if fecha_ingresada > fecha_actual:
                    print('\n** LA FECHA INGRESADA NO DEBE SER POSTERIOR A LA FECHA ACTUAL. INGRESA UNA FECHA VÁLIDA **')
                else:
                    break
            except Exception:
                print('\n** EL FORMATO DE FECHA NO ES VÁLIDO. INGRESA LA FECHA EN FORMATO DD/MM/YYYY **')
        try:
            with sqlite3.connect('C:/Users/betyh/Downloads/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveServicio, descripcion, costo FROM servicios WHERE estadoS = 1')
                datos_s = mi_cursor.fetchall()
                tabla = PrettyTable()
                tabla.field_names = ['CLAVE', 'DESCRIPCIÓN','COSTO']
                print('\n      Servicios Registrados')
                for dato in datos_s:
                    tabla.add_row(dato)
                print(tabla)
        except Error as e:
            print(e)
        except Exception:
            print(f'Se produjo el siguiente error: {sys.exc_info()[0]}')
        finally:
            conn.close()

        while True:

                servicio = input('\nIngrese la clave del servicio que desea solicitar: ')
                if servicio.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UN DATO O (SALIR) PARA VOLVER AL MENÚ PRINCIPAL **')
                    continue  # Volver al inicio del bucle
                with sqlite3.connect('C:/Users/betyh/Downloads/tallermecanico.db') as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute('SELECT ClaveServicio FROM servicios WHERE ClaveServicio = ? AND estadoS = 1', (servicio,))
                    servicio_existente = mi_cursor.fetchall()

                if not servicio_existente:
                    print('\n** LA CLAVE DE SERVICIO INGRESADA NO ESTÁ REGISTRADA. POR FAVOR, INGRESE UN DATO VÁLIDO **')
                    continue  # Volver al inicio del bucle

                servicio_n = input('\n¿Deseas agregar otro servicio? (S)i (N)o: ')
                if servicio_n.lower() != 's':
                    try:
                        with sqlite3.connect('C:/Users/betyh/Downloads/tallermecanico.db') as conn:
                            mi_cursor = conn.cursor()
                            valores = (fecha_ingresada, ClaveCliente, servicio)
                            mi_cursor.execute('INSERT INTO notas (fecha, ClaveCliente, ClaveServicio) VALUES (?,?,?)', valores)
                            return
                            print(f'Se asignó la clave {mi_cursor.lastrowid}')
                    except sqlite3.Error as e:
                        print(e)
                    finally:
                        conn.close()
                    agregar_nota = input('\n¿Deseas registrar otra nota? (S)i (N)o: ')
                    if agregar_nota.lower() != 's':
                       print('\n** NOTA(S) REGISTRADA(S) CORRECTAMENTE **')
                       break
                    else:
                       print('\n** NOTA(S) REGISTRADA(S) CORRECTAMENTE **')
                    break


**MENÚ CLIENTES**

In [7]:
def validar_correo(correo):
    patron = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return bool(re.match(patron, correo))

def validar_rfc(rfc):
    patron = r'^[A-Z]{4}[0-9]{6}[A-Z0-9]{3}$'
    return bool(re.match(patron, rfc))

def agregar_cliente():
    estadoC = True
    while True:
        print('\n   ════════════════════════════')
        print('    REGISTRA UN NUEVO CLIENTE')
        print('   ════════════════════════════')
        print('\nNota: Escriba (salir) si desea volver al menú clientes')
        while True:
            try:
                nombre = input('\nIngrese el nombre completo del cliente: ')
                if nombre.lower() == 'salir':
                    print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ CLIENTES **')
                    return
                if nombre.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UN DATO O (SALIR) PARA VOLVER AL MENÚ CLIENTES **')
                    continue
                else:
                    break
            except ValueError as e:
                print(e)
        while True:
            try:
                correo = input('\nIngrese el correo electrónico del cliente: ')
                if correo.lower() == 'salir':
                    print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ CLIENTES **')
                    return
                elif correo.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UN DATO O (SALIR) PARA VOLVER AL MENÚ CLIENTES **')
                    continue
                elif validar_correo(correo):
                    break
                else:
                    print('\n** EL CORREO PROPORCIONADO NO TIENE UN FORMATO VÁLIDO. INGRESELO NUEVAMENTE **')
                    continue
            except Error as e:
                print(e)
        while True:
            try:
                rfc = input('\nIngrese el rfc del cliente: ')
                if rfc.lower() == 'salir':
                    print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ PRINCIPAL **')
                    return
                elif rfc.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UN DATO O (SALIR) PARA VOLVER AL MENÚ PRINCIPAL **')
                    continue
                elif validar_rfc(rfc):
                    break
                else:
                    print('\n** EL RFC PROPORCIONADO NO TIENE UN FORMATO VÁLIDO. INGRESELO NUEVAMENTE **')
                    continue
            except Error as e:
                print(e)
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                datos = (nombre, correo, rfc, estadoC)
                mi_cursor.execute('INSERT INTO clientes (nombre,correo,rfc,estadoC) VALUES (?,?,?,?)', datos)
                print('\n** CLIENTE(S) REGISTRADO CORRECTAMENTE **')
                print(f'Clave asignada: {mi_cursor.lastrowid}')
        except Error as e:
            print(f'Se produjo el siguiente error: {e}')
        finally:
            conn.close()
        while True:
            agregar_cliente = input('\n¿Desea agregar otro cliente? (Si) (No): ')
            if agregar_cliente.lower() == 's':
                break
            elif agregar_cliente.lower() == 'n':
                print('\n** OPERACIÓN CANCELADA. VOLVIENDO AL MENÚ CLIENTES**')
                return
            else:
                print('\n** DATO NO VÁLIDO. INGRESE (S)I Ó (N)O **')
                continue

def suspender_cliente():
    while True:
        print('\n   ════════════════════════════')
        print('       SUSPENDER UN CLIENTE')
        print('   ════════════════════════════')
        print('\nNota: Escriba (salir) si desea volver al menú clientes')
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveCliente, nombre FROM clientes WHERE estadoC = True')
                datos = mi_cursor.fetchall()
                tabla = PrettyTable()
                tabla.field_names = ["CLAVE", "NOMBRE"]
                print('\n    Clientes Registrados')
                for dato in datos:
                    tabla.add_row(dato)
                print(tabla)
        except Error as e:
            print(e)
        except Exception:
            print(f'Se produjo el siguiente error: {sys.exc_info()[0]}')
        while True:
            try:
                clave_suspender = input('\nIngrese la clave del cliente a suspender: ')
                if clave_suspender.lower() == 'salir':
                    print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ CLIENTES **')
                    return
                if clave_suspender.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UN DATO O (SALIR) PARA VOLVER AL MENÚ CLIENTE **')
                    continue
                else:
                    clave_suspender = int(clave_suspender)
                    break
            except Exception:
                print(f'\n** DATO NO VÁLIDO. INGRESE LA CLAVE DE ALGÚN CLIENTE **')
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveCliente FROM clientes WHERE ClaveCliente = ? AND estadoC = True', (clave_suspender,))
                cliente_existente = mi_cursor.fetchall()
                if cliente_existente:
                    mi_cursor.execute('SELECT ClaveCliente, nombre, correo, rfc FROM clientes WHERE ClaveCliente = ?', (clave_suspender,))
                    datos = mi_cursor.fetchall()
                    tabla = PrettyTable()
                    tabla.field_names = ['CLAVE', 'NOMBRE','CORREO','RFC']
                    print('\n    Cliente a suspender')
                    for dato in datos:
                        tabla.add_row(dato)
                    print(tabla)
                    while True:
                        print('Nota: Ingrese la letra (S) si confirma la acción ó (N) si desea cancelar la operación')
                        confirmacion = input('\n¿Está seguro que desea suspender este registro? (S)I Ó (N)O: ')
                        if confirmacion.lower() == 's':
                            mi_cursor.execute('UPDATE clientes SET estadoC = False WHERE ClaveCliente = ?', (clave_suspender,))
                            print('\n** Cliente suspendido correctamente **')
                            break
                        elif confirmacion.lower() == 'n':
                            print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ CLIENTES **')
                            return
                        else:
                            print('\n** DATO NO VÁLIDO. INGRESE (S)I Ó (N)O **')
                else:
                    print('\n** EL CLIENTE NO EXISTE O SE ENCUENTRA SUSPENDIDO. **')
                    continue
        finally:
            conn.close()

def recuperar_cliente():
    while True:
        print('\n   ════════════════════════════')
        print('       RECUPERAR UN CLIENTE')
        print('   ════════════════════════════')
        print('\nNota: Escriba (salir) si desea volver al menú clientes')
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveCliente, nombre FROM clientes WHERE estadoC = False')
                datos = mi_cursor.fetchall()
                tabla = PrettyTable()
                tabla.field_names = ["CLAVE", "NOMBRE"]
                print('\n    Clientes Suspendidos')
                for dato in datos:
                    tabla.add_row(dato)
                print(tabla)
        except Error as e:
            print(e)
        except Exception:
            print(f'Se produjo el siguiente error: {sys.exc_info()[0]}')
        while True:
            try:
                clave_recuperar = input('\nIngrese la clave del cliente a recuperar: ')
                if clave_recuperar.lower() == 'salir':
                    print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ CLIENTES **')
                    return
                if clave_recuperar.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE. INGRESE UN DATO O (SALIR) PARA VOLVER AL MENÚ CLIENTE **')
                    continue
                else:
                    clave_recuperar = int(clave_recuperar)
                    break
            except Exception:
                print(f'\n** DATO NO VÁLIDO. INGRESE LA CLAVE DE ALGÚN CLIENTE **')
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveCliente FROM clientes WHERE ClaveCliente = ? AND estadoC = False', (clave_recuperar,))
                cliente_suspendido = mi_cursor.fetchall()
                if cliente_suspendido:
                    mi_cursor.execute('SELECT ClaveCliente, nombre, correo, rfc FROM clientes WHERE ClaveCliente = ?', (clave_recuperar,))
                    datos = mi_cursor.fetchall()
                    tabla = PrettyTable()
                    tabla.field_names = ['CLAVE', 'NOMBRE','CORREO','RFC']
                    print('\n    Cliente a recuperar')
                    for dato in datos:
                        tabla.add_row(dato)
                    print(tabla)
                    while True:
                        print('Nota: Ingrese la letra (S) si confirma la acción ó (N) si desea cancelar la operación y volver al menú')
                        confirmacion = input('\n¿Está seguro que desea recuperar este registro? (S)I Ó (N)O: ')
                        if confirmacion.lower() == 's':
                            mi_cursor.execute('UPDATE clientes SET estadoC = True WHERE ClaveCliente = ?', (clave_recuperar,))
                            print('\n** Cliente recuperado correctamente **')
                            break
                        elif confirmacion.lower() == 'n':
                            print('\n** OPERACION CANCELADA. VOLVIENDO AL MENÚ CLIENTES **')
                            return
                        else:
                            print('\n** DATO NO VÁLIDO. POR FAVOR, INGRESE (S)I Ó (N)O **')
                else:
                    print('\n** EL CLIENTE NO SE ENCUENTRA SUSPENDIDO. **')
                    continue
        finally:
            conn.close()

def listado_clientes_registrados():
    while True:
        print('\n-----------------------------------------')
        print(' SUBMENÚ LISTADO DE CLIENTES REGISTRADOS')
        print('-----------------------------------------')
        print('1. Ordenado por clave')
        print('2. Ordenado por nombre')
        print('3. Volver al menú anterior')
        opcion = int(input('\nIngresa el número de la operación que deseas realizar: '))
        encabezados = ['CLAVE', 'NOMBRE', 'CORREO', 'RFC']
        if opcion == 1:
            try:
                with sqlite3.connect('/content/tallermecanico.db') as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute('SELECT ClaveCliente, nombre, correo, rfc FROM clientes WHERE estadoC = True ORDER BY ClaveCliente')
                    datos = mi_cursor.fetchall()
                    tabla = PrettyTable()
                    tabla.field_names = (encabezados)
                    print(f'\n                    CLIENTES ACTIVOS')
                    for dato in datos:
                        tabla.add_row(dato)
                    print(tabla)
                    print('\n---------------------------------------')
                    print('           EXPORTAR REPORTE')
                    print('---------------------------------------')
                    print('1. Exportar reporte como archivo EXCEL')
                    print('2. Exportar reporte como archivo CSV')
                    print('3. Volver al menú de reportes')
                    exportar = int(input('\nIngresa el número de la operación que deseas realizar: '))
                    if exportar == 1:
                        fecha_reporte = datetime.now().strftime('%d_%m_%Y')
                        nombre_excel = f'ReporteClientesActivosPorClave_{fecha_reporte}.xlsx'
                        wb = Workbook()
                        hoja = wb.active
                        hoja.append(encabezados)
                        for dato in datos:
                            hoja.append(dato)
                        wb.save(nombre_excel)
                        print(f'\nInforme {nombre_excel} exportado correctamente')
                    elif exportar == 2:
                        fecha_reporte = datetime.now().strftime('%d_%m_%Y')
                        nombre_csv = f'ReporteClientesActivosPorClave_{fecha_reporte}.csv'
                        with open(nombre_csv, 'w', newline='') as reporte_csv:
                            grabador = csv.writer(reporte_csv)
                            grabador.writerow(encabezados)
                            grabador.writerows(datos)
                        print(f'\nInforme {nombre_csv} exportado correctamente')
                    elif exportar == 3:
                        break
            except Exception as e:
                print(f"Error: {e}")
        elif opcion == 2:
            try:
                with sqlite3.connect('/content/tallermecanico.db') as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute('SELECT ClaveCliente, nombre, correo, rfc FROM clientes WHERE estadoC = True ORDER BY nombre')
                    datos = mi_cursor.fetchall()
                    tabla = PrettyTable()
                    tabla.field_names = (encabezados)
                    print(f'\n                    CLIENTES ACTIVOS')
                    for dato in datos:
                        tabla.add_row(dato)
                    print(tabla)
                    print('\n---------------------------------------')
                    print('           EXPORTAR REPORTE')
                    print('---------------------------------------')
                    print('1. Exportar reporte como archivo EXCEL')
                    print('2. Exportar reporte como archivo CSV')
                    print('3. Volver al menú de reportes')
                    exportar = int(input('\nIngresa el número de la operación que deseas realizar: '))
                    if exportar == 1:
                        fecha_reporte = datetime.now().strftime('%d_%m_%Y')
                        nombre_excel = f'ReporteClientesActivosPorNombre_{fecha_reporte}.xlsx'
                        wb = Workbook()
                        hoja = wb.active
                        hoja.append(encabezados)
                        for dato in datos:
                            hoja.append(dato)
                        wb.save(nombre_excel)
                        print(f'\nInforme {nombre_excel} exportado correctamente')
                    elif exportar == 2:
                        fecha_reporte = datetime.now().strftime('%d_%m_%Y')
                        nombre_csv = f'ReporteClientesActivosPorNombre_{fecha_reporte}.csv'
                        with open(nombre_csv, 'w', newline='') as reporte_csv:
                            grabador = csv.writer(reporte_csv)
                            grabador.writerow(encabezados)
                            grabador.writerows(datos)
                        print(f'\nInforme {nombre_csv} exportado correctamente')
                    elif exportar == 3:
                        break
            except Exception as e:
                print(f"Error: {e}")
        elif opcion == 3:
            break

def consultas_reportes_clientes():
    while True:
        print('\n  ═══════════════════════════════════')
        print('   CONSULTAS Y REPORTES DE CLIENTES')
        print('  ═══════════════════════════════════')
        print('1. Listado de clientes registrados')
        print('2. Búsqueda por clave')
        print('3. Búsqueda por nombre')
        print('4. Volver al menú clientes')
        consulta = int(input('\nIngresa el número de la operación que desea realizar: '))
        if 1 <= consulta <= 4:
            if consulta == 1:
                listado_clientes_registrados()
            elif consulta == 2:
              while True:
                try:
                  with sqlite3.connect('/content/tallermecanico.db') as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute('SELECT ClaveCliente, nombre FROM clientes ORDER BY ClaveCliente')
                    datos_clientes = mi_cursor.fetchall()
                    tabla_clientes = PrettyTable()
                    tabla_clientes.field_names = ['CLAVE', 'NOMBRE']
                    for cliente in datos_clientes:
                      tabla_clientes.add_row(cliente)
                    print(tabla_clientes)
                    clave_consultar = input('\nIngrese la clave del cliente a consultar (o escriba "salir" para volver): ')
                    if clave_consultar.lower() == 'salir':
                      break
                    clave_consultar = int(clave_consultar)
                    mi_cursor.execute('SELECT ClaveCliente, nombre, correo, rfc FROM clientes WHERE ClaveCliente = ?', (clave_consultar,))
                    datos_cliente_consultado = mi_cursor.fetchall()
                    tabla_cliente_consultado = PrettyTable()
                    tabla_cliente_consultado.field_names = ['CLAVE', 'NOMBRE', 'CORREO', 'RFC']
                    if datos_cliente_consultado:
                      print(f'\n    Cliente Encontrado')
                      for dato in datos_cliente_consultado:
                        tabla_cliente_consultado.add_row(dato)
                        print(tabla_cliente_consultado)
                      break
                    else:
                      print(f'\n** NO SE ENCONTRÓ UN REGISTRO ASOCIADO A LA CLAVE {clave_consultar}')
                except Exception:
                  print(f'\n** DATO NO VÁLIDO. INGRESE LA CLAVE DE ALGÚN CLIENTE **')
            elif consulta == 3:
                while True:
                    print('\nNota: Escriba (salir) si desea volver a consultas y reportes de clientes')
                    try:
                        nombre_consultar = input('\nIngrese el nombre completo del cliente a consultar: ')
                        if nombre_consultar.lower() == 'salir':
                            break
                        else:
                            with sqlite3.connect('/content/tallermecanico.db') as conn:
                                mi_cursor = conn.cursor()
                                datos = {'nombre': nombre_consultar.lower()}
                                mi_cursor.execute('SELECT ClaveCliente, nombre, correo, rfc FROM clientes WHERE LOWER (nombre) = :nombre', datos)
                                datos1 = mi_cursor.fetchall()
                                tabla = PrettyTable()
                                tabla.field_names = ['CLAVE', 'NOMBRE', 'CORREO','RFC']
                                if datos1:
                                    print(f'\n    Cliente Encontrado')
                                    for dato in datos1:
                                        tabla.add_row(dato)
                                    print(tabla)
                                    break
                                else:
                                    print(f'\n** NO SE ENCONTRÓ UN REGISTRO ASOCIADO AL {nombre_consultar}')
                    except Exception:
                        print(f'\n** DATO NO VÁLIDO. INGRESE EL NOMBRE DE ALGÚN CLIENTE **')
            elif consulta == 4:
                return

def menu_clientes():
    while True:
        print('\n--------------------------')
        print('      MENÚ CLIENTES      ')
        print('--------------------------')
        print('1. Agregar cliente       ')
        print('2. Suspender cliente     ')
        print('3. Recuperar cliente     ')
        print('4. Consultas y reportes   ')
        print('5. Volver al menú principal   ')
        while True:
            try:
                opcion = int(input('\nIngresa el número de la operación que deseas realizar: '))
                if 1 <= opcion <= 5:
                    if opcion == 1:
                        if not agregar_cliente():
                            break
                    elif opcion == 2:
                        if not suspender_cliente():
                            break
                    elif opcion == 3:
                        if not recuperar_cliente():
                            break
                    elif opcion == 4:
                        if not consultas_reportes_clientes():
                            break
                    elif opcion == 5:
                        return
                else:
                    raise ValueError
            except ValueError:
                print('\n** OPCIÓN NO VÁLIDA. POR FAVOR, REGISTRE EL NÚMERO DE ALGUNA OPCIÓN MOSTRADA **')


**MENÚ SERVICIOS**

In [24]:
def agregar_servicio():
    while True:
        print('\n   ════════════════════════════')
        print('    AGREGAR UN NUEVO SERVICIO')
        print('   ════════════════════════════')
        print('\nNota: Escriba (salir) si desea volver al menú clientes')
        while True:
            try:
                descripcion = input('\nIngrese el nombre del servicio: ')
                if descripcion.lower() == 'salir':
                    print('\n** OPERACION CANCELADA, VOLVIENDO AL MENÚ PRINCIPAL **')
                    return
                if descripcion.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE, INGRESE UNA DATO O (SALIR) PARA VOLVER AL MENÚ PRINCIPAL **')
                    continue
                else:
                    break
            except ValueError as e:
                print(e)
        while True:
            try:
                costo = input('\nIngrese el costo del servicio: ')
                if costo.lower() == 'salir':
                    print('\n** OPERACION CANCELADA, VOLVIENDO AL MENÚ PRINCIPAL **')
                    return
                elif costo.strip() == '':
                    print('\n** EL DATO NO PUEDE OMITIRSE, INGRESE UNA DATO O (SALIR) PARA VOLVER AL MENÚ PRINCIPAL **')
                    continue
                costo = float(costo)
                if costo <= 0.0:
                    print('\n** EL COSTO DEL SERVICIO DEBE SER MAYOR A 0.0, INGRESE UN COSTO VÁLIDO **')
                    continue
                else:
                    break
            except ValueError:
                print('\n** DATO NO VÁLIDO, INGRESE UN COSTO **')
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                datos = (descripcion,costo)
                mi_cursor.execute('INSERT INTO servicios (descripcion, costo, estadoS) VALUES (?,?, 1)', datos)
                print('\n**Servicio registrado **')
                print(f'Clave asignada: {mi_cursor.lastrowid}')
        except Error as e:
            print(f'Se produjo el siguiente error: {e}')
        finally:
            conn.close()
        try:
            while True:
                agregar_servicio = input('\n¿Desea agregar otro servicio? (S)i (N)o: ')
                if agregar_servicio.lower() == 's':
                    break
                elif agregar_servicio.lower() == 'n':
                    print('\n** SERVICIO(S) REGISTRADO CORRECTAMENTE **')
                    raise StopIteration
                else:
                    print('\n** DATO NO VÁLIDO, INGRESE (S)I Ó (N)O **')
                    continue
        except StopIteration:
            break

def suspender_servicio():
    while True:
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveServicio, descripcion FROM servicios WHERE estadoS = 1')
                datos = mi_cursor.fetchall()
                if not datos:
                    print('\nNo hay servicios disponibles.')
                else:
                    tabla = PrettyTable()
                    tabla.field_names = ["Clave Servicio", "Nombre"]
                    for dato in datos:
                        tabla.add_row(dato)
                    print('\n       Servicios Suspendidos ')
                    print(tabla)
        except Exception as e:
            print(f'Se produjo el siguiente error: {e}')
        print('\n   SUSPENDER UN SERVICIO')
        print('\nNota: Escriba (salir) si desea volver al menú principal')
        idservicio_suspender = input('Ingresa la clave del servicio a suspender: ')

        if idservicio_suspender.lower() == 'salir':
            return
        try:
            idservicio_suspender = int(idservicio_suspender)
        except ValueError:
            print('\n** ERROR, ingrese un número de servicio válido o escriba "salir" para volver al menú principal **')
            continue
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveServicio FROM servicios WHERE ClaveServicio = ?', (idservicio_suspender,))
                servicio_existente = mi_cursor.fetchall()

                if not servicio_existente:
                    print('\n** ERROR, no se encontró un servicio con esa clave **')
                    continue

                confirmacion = input('\n¿Desea suspender este servicio? (S)i (N)o: ')
                if confirmacion.lower() == 's':
                    mi_cursor.execute('UPDATE servicios SET estadoS = 0 WHERE ClaveServicio = ?', (idservicio_suspender,))
                    conn.commit()
                    print('\n** Servicio suspendido correctamente **')
                elif confirmacion.lower() == 'n':
                    print('\n** OPERACIÓN CANCELADA, VOLVIENDO AL MENÚ PRINCIPAL **')
                else:
                    print('\n** ERROR, entrada no válida. Por favor, ingrese "S" o "N" **')
        except sqlite3.Error as e:
            print(f'Se produjo el siguiente error: {e}')
        break

def recuperar_servicio():
    while True:
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveServicio, descripcion FROM servicios WHERE estadoS = 0')  #
                datos = mi_cursor.fetchall()

                if not datos:
                    print('\nNo hay servicios suspendidos.')
                else:
                    tabla = PrettyTable()
                    tabla.field_names = ["Clave Servicio", "Nombre"]
                    for dato in datos:
                        tabla.add_row(dato)
                    print('\n       Servicios Suspendidos ')
                    print(tabla)
        except Exception as e:
            print(f'Se produjo el siguiente error: {e}')

        print('\n   RECUPERAR UN SERVICIO')
        print('\nNota: Escriba (salir) si desea volver al menú principal')
        idservicio_recuperar = input('Ingresa la clave del servicio a recuperar: ')

        if idservicio_recuperar.lower() == 'salir':
            return
        try:
            idservicio_recuperar = int(idservicio_recuperar)
        except ValueError:
            print('\n** ERROR, ingrese un número de servicio válido o escriba "salir" para volver al menú principal **')
            continue
        try:
            with sqlite3.connect('/content/tallermecanico.db') as conn:
                mi_cursor = conn.cursor()
                mi_cursor.execute('SELECT ClaveServicio FROM servicios WHERE ClaveServicio = ?', (idservicio_recuperar,))
                servicio_existente = mi_cursor.fetchall()

                if not servicio_existente:
                    print('\n** ERROR, no se encontró un servicio con esa clave **')
                    continue
                confirmacion = input('\n¿Desea recuperar este servicio? (S)i (N)o: ')
                if confirmacion.lower() == 's':
                    mi_cursor.execute('UPDATE servicios SET estadoS = 1 WHERE ClaveServicio = ?', (idservicio_recuperar,))
                    conn.commit()
                    print('\n** Servicio recuperado correctamente **')
                elif confirmacion.lower() == 'n':
                    print('\n** OPERACIÓN CANCELADA, VOLVIENDO AL MENÚ PRINCIPAL **')
                else:
                    print('\n** ERROR, entrada no válida. Por favor, ingrese "S" o "N" **')
        except sqlite3.Error as e:
            print(f'Se produjo el siguiente error: {e}')
        break

def listado_servicios_registrados():
    while True:
        print('---------------------------------------')
        print('SUBMENÚ LISTADO DE SERVICIOS REGISTRADOS')
        print('---------------------------------------')
        print('1. Ordenado por clave')
        print('2. Ordenado por nombre de servicio')
        print('3. Volver al menú anterior')
        opcion = int(input('\nIngresa el número de la operación que deseas realizar: '))
        encabezados = ['Clave', 'Nombre de Servicio', 'Descripción', 'Costo']
        if opcion == 1:
            try:
                with sqlite3.connect('/content/tallermecanico.db') as conn:
                  mi_cursor = conn.cursor()
                  mi_cursor.execute('SELECT ClaveServicio, descripcion, costo FROM servicios ORDER BY ClaveServicio')
                  datos = mi_cursor.fetchall()
                  if datos:
                    tabla = PrettyTable()
                    tabla.field_names = ["ClaveServicio", "Descripción", "Costo"]
                    for ClaveServicio, descripcion, costo in datos:
                      tabla.add_row([ClaveServicio, descripcion, costo])
                    print(tabla)
                    print("\nSERVICIOS REGISTRADOS")
                  else:
                    print("\n** NO HAY SERVICIOS REGISTRADOS **")

                  print('\n---------------------------------------')
                  print('           EXPORTAR REPORTE')
                  print('---------------------------------------')
                  print('1. Exportar reporte como archivo EXCEL')
                  print('2. Exportar reporte como archivo CSV')
                  print('3. Volver al menú de reportes')
                  exportar = int(input('\nIngresa el número de la operación que deseas realizar: '))
                  if exportar == 1:
                        fecha_reporte = datetime.now().strftime('%m_%d_%Y')
                        nombre_excel = f'ReporteServiciosPorClave_{fecha_reporte}.xlsx'
                        wb = Workbook()
                        hoja = wb.active
                        hoja.append(encabezados)
                        for dato in datos:
                            hoja.append(dato)
                        wb.save(nombre_excel)
                        print(f'\nInforme {nombre_excel} exportado correctamente')
                  elif exportar == 2:
                        fecha_reporte = datetime.now().strftime('%m_%d_%Y')
                        nombre_csv = f'ReporteServiciosPorClave_{fecha_reporte}.csv'
                        with open(nombre_csv, 'w', newline='') as reporte_csv:
                            grabador = csv.writer(reporte_csv)
                            grabador.writerow(encabezados)
                            grabador.writerows(datos)
                        print(f'\nInforme {nombre_csv} exportado correctamente')
                  elif exportar == 3:
                        break
            except Exception as e:
                print(f"Error: {e}")
        elif opcion == 2:
            try:
                with sqlite3.connect('/content/tallermecanico.db') as conn:
                  mi_cursor = conn.cursor()
                  mi_cursor.execute('SELECT ClaveServicio, descripcion, costo FROM servicios ORDER BY descripcion')
                  datos = mi_cursor.fetchall()
                  if datos:
                    tabla = PrettyTable()
                    tabla.field_names = ["ClaveServicio", "Descripción", "Costo"]
                    for ClaveServicio, descripcion, costo in datos:
                      tabla.add_row([ClaveServicio, descripcion, costo])
                    print(tabla)
                    print("\nSERVICIOS REGISTRADOS")
                  else:
                    print("\n** NO HAY SERVICIOS REGISTRADOS **")
                print('\n---------------------------------------')
                print('           EXPORTAR REPORTE')
                print('---------------------------------------')
                print('1. Exportar reporte como archivo EXCEL')
                print('2. Exportar reporte como archivo CSV')
                print('3. Volver al menú de reportes')
                exportar = int(input('\nIngresa el número de la operación que deseas realizar: '))
                if exportar == 1:
                        fecha_reporte = datetime.now().strftime('%m_%d_%Y')
                        nombre_excel = f'ReporteServiciosPorNombre_{fecha_reporte}.xlsx'
                        wb = Workbook()
                        hoja = wb.active
                        hoja.append(encabezados)
                        for dato in datos:
                            hoja.append(dato)
                        wb.save(nombre_excel)
                        print(f'\nInforme {nombre_excel} exportado correctamente')
                elif exportar == 2:
                        fecha_reporte = datetime.now().strftime('%m_%d_%Y')
                        nombre_csv = f'ReporteServiciosPorNombre_{fecha_reporte}.csv'
                        with open(nombre_csv, 'w', newline='') as reporte_csv:
                            grabador = csv.writer(reporte_csv)
                            grabador.writerow(encabezados)
                            grabador.writerows(datos)
                        print(f'\nInforme {nombre_csv} exportado correctamente')
                elif exportar == 3:
                        break
            except Exception as e:
                print(f"Error: {e}")
        elif opcion == 3:
            break

def consultas_reportes_servicios():
    while True:
        print('\n  ═══════════════════════════════════')
        print('   CONSULTAS Y REPORTES DE SERVICIOS')
        print('  ═══════════════════════════════════')
        print('1. Listado de servicios registrados')
        print('2. Búsqueda por clave')
        print('3. Búsqueda por nombre')
        print('4. Volver al menú servicios')
        consulta = int(input('\nIngresa el número de la operación que desea realizar: '))
        if 1 <= consulta <= 4:
            if consulta == 1:
                listado_servicios_registrados()
            elif consulta == 2:
              while True:
                try:
                  with sqlite3.connect('/content/tallermecanico.db') as conn:
                    mi_cursor = conn.cursor()
                    mi_cursor.execute('SELECT ClaveServicio, descripcion FROM servicios ORDER BY ClaveServicio')
                    datos_servicios = mi_cursor.fetchall()
                    tabla_servicios = PrettyTable()
                    tabla_servicios.field_names = ['CLAVE', 'DESCRIPCION']
                    for servicio in datos_servicios:
                      tabla_servicios.add_row(servicio)
                    print(tabla_servicios)
                    clave_consultar = input('\nIngrese la clave del servicio a consultar (o escriba "salir" para volver): ')
                    if clave_consultar.lower() == 'salir':
                      break
                    clave_consultar = int(clave_consultar)
                    mi_cursor.execute('SELECT ClaveServicio, descripcion, costo FROM servicios WHERE ClaveServicio = ?', (clave_consultar,))
                    datos_servicio_consultado = mi_cursor.fetchall()
                    datos_servicio_consultado = PrettyTable()
                    datos_servicio_consultado.field_names = ['CLAVE', 'DESCRIPCION', 'COSTO']
                    if datos_servicio_consultado:
                      print(f'\n    Servicio Encontrado')
                      for dato in datos_servicio_consultado:
                        datos_servicio_consultado.add_row(dato)
                        print(datos_servicio_consultado)
                      break
                    else:
                      print(f'\n** NO SE ENCONTRÓ UN REGISTRO ASOCIADO A LA CLAVE {clave_consultar}')
                except Exception:
                  print(f'\n** DATO NO VÁLIDO. INGRESE LA CLAVE DE ALGÚN CLIENTE **')
            elif consulta == 3:
                while True:
                    print('\nNota: Escriba (salir) si desea volver a consultas y reportes de servicios')
                    try:
                        nombre_consultar = input('\nIngrese el nombre del servicio a consultar: ')
                        if nombre_consultar.lower() == 'salir':
                            break
                        else:
                            with sqlite3.connect('/content/tallermecanico.db') as conn:
                                mi_cursor = conn.cursor()
                                datos = {'nombre': nombre_consultar.lower()}
                                mi_cursor.execute('SELECT ClaveServicio, descripcion, costo FROM servicios WHERE LOWER (descripcion) = :nombre', datos)
                                datos1 = mi_cursor.fetchall()
                                tabla = PrettyTable()
                                tabla.field_names = ['CLAVE', 'DESCRIPCION', 'COSTO']
                                if datos1:
                                    print(f'\n    Servicio Encontrado')
                                    for dato in datos1:
                                        tabla.add_row(dato)
                                    print(tabla)
                                    break
                                else:
                                    print(f'\n** NO SE ENCONTRÓ UN REGISTRO ASOCIADO AL {nombre_consultar}')
                    except Exception:
                        print(f'\n** DATO NO VÁLIDO. INGRESE EL NOMBRE DE ALGÚN SERVICIO **')
            elif consulta == 4:
                return

def menu_servicios():
    while True:
        print('\n---------------------------------------')
        print('  BIENVENIDO AL MENU DE SERVICIOS   ')
        print('---------------------------------------')
        print('1. Agregar un servicio')
        print('2. Suspender un servicio')
        print('3. Recuperar un servicio')
        print('4. Consultas y reportes')
        print('5. Regresar al menú principal')
        opcion = input('\nIngrese el número de la opcion a la que desea ingresar: ')
        if opcion.isdigit():
            opcion = int(opcion)
            if opcion == 1:
                agregar_servicio()
            elif opcion == 2:
                suspender_servicio()
            elif opcion == 3:
                recuperar_servicio()
            elif opcion == 4:
                consultas_reportes_servicios()
            elif opcion == 5:
                respuesta = input("¿Desea salir? (S/N) ")
                if respuesta.upper() == 'S':
                    menu_principal()
            else:
                print("Opción no válida. Por favor, elige una opción válida.")
        else:
            print('\nOpción no válida. Por favor, elige una opción válida.')

**MENÚ PRINCIPAL**

In [23]:
def menu_principal():
    while True:
        print('---------------------------------------')
        print('  BIENVENIDO AL MENU PRINCIPAL   ')
        print('---------------------------------------')
        print('1. Notas')
        print('2. Clientes')
        print('3. Servicios')
        print('4. Salir')
        opcion = input('Ingrese el número del menú al que desea ingresar: ')
        if opcion.isdigit():
            opcion = int(opcion)
            if opcion == 1:
                print("Entraste al menú de notas")
            elif opcion == 2:
                menu_clientes()
            elif opcion == 3:
                print("Entraste a Servicios")
                menu_servicios()
            elif opcion == 4:
                respuesta = input("¿Desea salir? (S/N) ")
                if respuesta.lower() == 's':
                    break
            else:
                print("Opción no válida. Por favor, elige una opción válida.")
        else:
            print('\nOpción no válida. Por favor, elige una opción válida.')

menu_principal()


---------------------------------------
  BIENVENIDO AL MENU PRINCIPAL   
---------------------------------------
1. Notas
2. Clientes
3. Servicios
4. Salir
Ingrese el número del menú al que desea ingresar: 4
¿Desea salir? (S/N) s
